# Testing per image strip processing

This notebook is aimed at prototyping the framework for slicing/windowing a large image into small windows and then re-constructing it efficiently.

In [1]:
import os.path as op
import ogr
import gdal
import rasterio
from skimage.transform import downscale_local_mean

from divdet.utils_inference import (iter_grouper, get_slice_bounds,
                                    yield_windowed_reads_numpy)
from divdet.inference.inference_runner import pred_generator

image_fpath_orig = '/Users/wronk/Data/divot_detect/B04_011293_1265_XN_53S071W.tiff'
output_fpath = '/Users/wronk/Data/divot_detect/B04_011293_1265_XN_53S071W_warp.tiff'

# One message per image
message = dict(scales=[2, 1], 
               window_size=(1024, 1024),
               min_window_overlap=(256, 256),
               endpoint='my_endpoint',
               batch_size=3)

## Reprojection

In [2]:
image_fpath = op.join(op.splitext(image_fpath_orig)[0] + '_warp' + 
                      op.splitext(image_fpath_orig)[1])
print(f'Reprojecting image and saving to {image_fpath}.')

with rasterio.open(image_fpath_orig) as temp_img:
    center_lat = temp_img.lnglat()[1]  # Raster center
eqc_proj = f'+proj=eqc +lat_ts={center_lat} +lat_0=0 +lon_0=180 +x_0=0 +y_0=0 +a=3396190 +b=3396190 +units=m +no_defs'
gdal.Warp(image_fpath, image_fpath_orig, dstSRS=eqc_proj)
print('Reprojection complete.')

Reprojecting image and saving to /Users/wronk/Data/divot_detect/B04_011293_1265_XN_53S071W_warp.tiff.
Reprojection complete.


## Slice image

In [3]:
preds = []
with rasterio.open(image_fpath) as dataset:
    image = dataset.read(1)

    for scale in message['scales']:
        print('Processing at downscale {scale}')

        # Rescale image, round, and convert back to orig datatype
        scaled_image = downscale_local_mean(
            image, (scale, scale)).round().astype(image.dtype)
        #image_size = (scaled_image.shape[1], scaled_image.shape[0])

        # Calculate slice bounds and create generator
        slice_bounds = get_slice_bounds(
            scaled_image.shape, slice_size=message.get('window_size'),
            min_window_overlap=message.get('min_window_overlap'))

        slice_gen = yield_windowed_reads_numpy(scaled_image, slice_bounds)
        
        # Calculate slice bounds and create generator
        pred_gen = pred_generator(slice_gen, message.get('endpoint'), 
                                  message.get('batch_size'))

        preds.extend(list(pred_gen))

Processing at downscale {scale}


0it [00:00, ?it/s]
  0%|          | 0/35 [00:00<?, ?it/s]

> /Users/wronk/Builds/divdet/divdet/inference/inference_runner.py(143)pred_generator()
    142         import ipdb; ipdb.set_trace()
--> 143         resp = requests.post(endpoint, data=payload)
    144         preds = json.loads(resp.content)['predictions']

ipdb> payload
'{"instances": [{"b64": "iVBORw0KGgoAAAANSUhEUgAABAAAAAQACAAAAABadnRfAAEAAElEQVR4nOz9ya51W3Yf-I1Zrnrttet99im-81W3insjgsEgGTRJOZXKNFIppSEkBMMJw25ky_BDuOuG38JNt91IwBVspyQqRIoMxo1bfvWp9tnlqotZZjcNG5mQxCIiOH9PsIEN_DHGmGPNCeA4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4juM4ju

ipdb> len(payload['instances'])
*** TypeError: string indices must be integers
ipdb> len(payload)
2173642
ipdb> len(instances)
3
ipdb> instances[1]
{'b64': 'iVBORw0KGgoAAAANSUhEUgAABAAAAAQACAAAAABadnRfAAEAAElEQVR4nOz9x65uXZaY6Y05pl32M3uffc75bURkkkmyCFaRTJrMKhlQhJq6CVVTkDqFglq6AV2YWgUIEkWTmZEZ8btj9t6fWXbaMdRgQ4BAoKpYaSIi53MFq_VirLWmAaiqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq

ipdb> q


BdbQuit: 